In [1]:
!pip install flair &> /dev/null

In [2]:
from glob import glob
dataset = glob("../input/twitter-loc/gold-random-json/*")

## Data Preprocessing

In [3]:
import json
from itertools import product
from flair.data import Sentence
from flair.tokenization import TokenizerWrapper
from nltk import wordpunct_tokenize

train_corpus = []
dev_corpus = []

for folder,corpus_type in product(dataset,["train","dev"]):
    for line in open(folder + '/' + corpus_type + '.jsonl'):
        obj = json.loads(line)
        sentence = Sentence(obj["text"],use_tokenizer = TokenizerWrapper(wordpunct_tokenize))

        for span_labels in obj["location_mentions"]:
            s,e = span_labels["start_offset"],span_labels["end_offset"]
            s_tok,e_tok = -1,-1
            for i,x in zip(range(len(sentence)),sentence):
                if s <= x.start_pos and x.end_pos <= e :
                    s_tok = s_tok if s_tok != -1 else i
                    e_tok = i
            try:
                sentence[s_tok:e_tok + 1].add_label("ner",span_labels["type"])  
            except:
                pass
        globals()[corpus_type + '_corpus'].append(sentence)
    
    

## Training

In [4]:
from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings,FlairEmbeddings,StackedEmbeddings,TransformerWordEmbeddings
from flair.trainers import ModelTrainer
from flair.data import Corpus

corpus = Corpus(train_corpus,dev_corpus) # store train and test data
corpus.downsample(0.35) #downsample to 35% 

### GLOVE + BiLSTM + CRF

In [5]:
label_type = 'ner' #Name-Entity Recognition (NER): It can recognise whether a 
                    #word represents a person, location or names in the text.


label_dict = corpus.make_label_dictionary(label_type=label_type) #Create a label dictionary from ner and data
print(label_dict)

embedding_types = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]
#The three embedding models will be concatenated and should give state of the art results.
#Document Embeddings generate one embedding for an entire text. 
#The produced embeddings are PyTorch vectors. 
#There are two different methods using the word embeddings to obtain a document embedding

embeddings = StackedEmbeddings(embeddings=embedding_types)

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

trainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.08,
              mini_batch_size=32,
              max_epochs=16)

2022-11-14 10:01:35,781 Computing label dictionary. Progress:


4534it [00:00, 51169.32it/s]

2022-11-14 10:01:35,878 Dictionary created for label 'ner' with 13 values: Country (seen 1618 times), State (seen 1338 times), City/town (seen 1103 times), Island (seen 324 times), County (seen 90 times), Human-made Point-of-Interest (seen 64 times), District (seen 54 times), Natural Point-of-Interest (seen 42 times), Continent (seen 27 times), Neighborhood (seen 19 times), Road/street (seen 19 times), Other locations (seen 15 times)
Dictionary with 13 tags: <unk>, Country, State, City/town, Island, County, Human-made Point-of-Interest, District, Natural Point-of-Interest, Continent, Neighborhood, Road/street, Other locations


2022-11-14 10:01:36,064 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp9e7sa6ox


100%|██████████| 160000128/160000128 [00:04<00:00, 37761289.26B/s]

...

2022-11-14 10:08:42,895 Evaluating as a multi-label problem: False
2022-11-14 10:08:42,910 DEV : loss 0.05537836253643036 - f1-score (micro avg)  0.8373


2022-11-14 10:08:42,992 BAD EPOCHS (no improvement): 0
2022-11-14 10:08:42,994 saving best model
2022-11-14 10:08:46,969 ----------------------------------------------------------------------------------------------------
2022-11-14 10:08:46,971 loading file resources/taggers/sota-ner-flair/best-model.pt
2022-11-14 10:08:48,518 SequenceTagger predicts: Dictionary with 51 tags: O, S-Country, B-Country, E-Country, I-Country, S-State, B-State, E-State, I-State, S-City/town, B-City/town, E-City/town, I-City/town, S-Island, B-Island, E-Island, I-Island, S-County, B-County, E-County, I-County, S-Human-made Point-of-Interest, B-Human-made Point-of-Interest, E-Human-made Point-of-Interest, I-Human-made Point-of-Interest, S-District, B-District, E-District, I-District, S-Natural Point-of-Interest, B-Natural Point-of-Interest, E-Natural Point-of-Interest, I-Natural Point-of-Interest, S-Continent, B-Continent, E-Continent, I-Continent, S-Neighborhood, B-Neighborhood, E-Neighborhood, I-Neighborhoo

100%|██████████| 16/16 [00:03<00:00,  4.32it/s]

2022-11-14 10:08:52,665 Evaluating as a multi-label problem: False
2022-11-14 10:08:52,679 0.8385	0.8175	0.8279	0.7455
2022-11-14 10:08:52,680 
Results:
- F-score (micro) 0.8279
- F-score (macro) 0.393
- Accuracy 0.7455

By class:
                              precision    recall  f1-score   support

                     Country     0.9453    0.9268    0.9360       205
                   City/town     0.6871    0.8000    0.7393       140
                       State     0.8824    0.8219    0.8511       146
                      Island     0.8485    0.8485    0.8485        33
                      County     0.7143    0.8333    0.7692         6
Human-made Point-of-Interest     0.0000    0.0000    0.0000        10
   Natural Point-of-Interest     0.0000    0.0000    0.0000         7
                   Continent     0.6667    0.5000    0.5714         4
                    District     0.0000    0.0000    0.0000         3
             Other locations     0.0000    0.0000    0.0000         

{'test_score': 0.8278985507246378,
 'dev_score_history': [0.5379959650302621,
  0.7531436135009927,
  0.779016393442623,
  0.7899686520376177,
  0.7979539641943734,
  0.806366047745358,
  0.8236040609137055,
  0.8140520896426408,
  0.7944636678200693,
  0.8309114927344782,
  0.8092409240924092,
  0.8125819134993447,
  0.8347266881028939,
  0.8254545454545456,
  0.8313773934527487,
  0.8372686662412252],
 'train_loss_history': [0.2619265969410788,
  0.12838247269544426,
  0.10402785370448052,
  0.09264882158382008,
  0.0852409312038254,
  0.08006780225204141,
  0.07475391906159923,
  0.07143794368417215,
  0.06782041281350115,
  0.06547560123382304,
  0.0631039529859215,
  0.06136859215317284,
  0.0587967874320589,
  0.05585249289286439,
  0.05542128599663147,
  0.05436841110678481],
 'dev_loss_history': [0.15231506526470184,
  0.09187287092208862,
  0.0807940810918808,
  0.07420500367879868,
  0.06877637654542923,
  0.06611115485429764,
  0.06359512358903885,
  0.06726896017789841,
  0

In [6]:
# del tagger,trainer,embeddings

### Transformer

In [7]:
# label_type = 'ner'

# label_dict = corpus.make_label_dictionary(label_type=label_type)
# print(label_dict)

# embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
#                                        layers="-1",
#                                        subtoken_pooling="first",
#                                        fine_tune=True,
#                                        use_context=True,
#                                        )

# tagger = SequenceTagger(hidden_size=256,
#                         embeddings=embeddings,
#                         tag_dictionary=label_dict,
#                         tag_type='ner',
#                         use_crf=False,
#                         use_rnn=False,
#                         reproject_embeddings=False,
#                         )

# trainer = ModelTrainer(tagger, corpus)

# trainer.fine_tune('resources/taggers/sota-ner-flair',
#               learning_rate=0.00085,
#               mini_batch_size=16,
#               max_epochs=8)

## INFERENCE

In [8]:
model = SequenceTagger.load('./resources/taggers/sota-ner-flair/final-model.pt') #loading model

2022-11-14 10:08:52,724 loading file ./resources/taggers/sota-ner-flair/final-model.pt
2022-11-14 10:08:54,297 SequenceTagger predicts: Dictionary with 51 tags: O, S-Country, B-Country, E-Country, I-Country, S-State, B-State, E-State, I-State, S-City/town, B-City/town, E-City/town, I-City/town, S-Island, B-Island, E-Island, I-Island, S-County, B-County, E-County, I-County, S-Human-made Point-of-Interest, B-Human-made Point-of-Interest, E-Human-made Point-of-Interest, I-Human-made Point-of-Interest, S-District, B-District, E-District, I-District, S-Natural Point-of-Interest, B-Natural Point-of-Interest, E-Natural Point-of-Interest, I-Natural Point-of-Interest, S-Continent, B-Continent, E-Continent, I-Continent, S-Neighborhood, B-Neighborhood, E-Neighborhood, I-Neighborhood, S-Road/street, B-Road/street, E-Road/street, I-Road/street, S-Other locations, B-Other locations, E-Other locations, I-Other locations, <START>


In [9]:
folder_to_check = '../input/twitter-loc/gold-random-json/kerala_floods_2018/' # END PATH WITH / symbol 

import os

for file in glob(folder_to_check + "*"):
    if "train" not in file:
        continue
    for line in open(file,'r'):
        obj = json.loads(line)
        s = Sentence(obj["text"],use_tokenizer = TokenizerWrapper(wordpunct_tokenize))
        model.predict(s) #model is being used to predict on sentence s
        outp =  {}
        outp["tweet_id"] = obj["tweet_id"]
        outp["location_mentions"] = []
        for e in s.labels:
            outp["location_mentions"].append({"text":e.data_point.text,"start_offset":e.data_point.start_position,"end_offset":e.data_point.end_position})
        with open("prediction.jsonl",'a') as out:
            json.dump(json.dumps(outp),out)
            out.write('\n')

In [10]:
numobs = 5

print("\033[1m Predictions \033[0m")
i = 0

for line in open("prediction.jsonl",'r'):
    print(json.loads(line))
    print()
    i += 1
    if i == numobs:
        break

print('\033[1m Ground Truth \033[0m')
i = 0

for line in open(folder_to_check + "train.jsonl"):
    #print(line)
    i += 1
    if i == numobs:
        break

 Predictions 
{"tweet_id": "1032071697221005312", "location_mentions": [{"text": "Kerala", "start_offset": 58, "end_offset": 64}, {"text": "Kerala", "start_offset": 204, "end_offset": 210}, {"text": "Karnataka", "start_offset": 223, "end_offset": 232}]}

{"tweet_id": "1034334390195822592", "location_mentions": []}

{"tweet_id": "1030975049510137856", "location_mentions": [{"text": "Kerala", "start_offset": 44, "end_offset": 50}]}

{"tweet_id": "1034481352341692417", "location_mentions": [{"text": "Kerala", "start_offset": 107, "end_offset": 113}, {"text": "India", "start_offset": 259, "end_offset": 264}]}

{"tweet_id": "1031529205673078784", "location_mentions": [{"text": "Kerala", "start_offset": 0, "end_offset": 6}, {"text": "india", "start_offset": 93, "end_offset": 98}, {"text": "kerala", "start_offset": 100, "end_offset": 106}, {"text": "Kerala", "start_offset": 208, "end_offset": 214}]}

 Ground Truth 
